# DATA PREPARATION STARTING FROM DATAFRAMES

In [1]:
import itertools
import pathlib
import pickle

import numpy as np
import pandas as pd
# import torch

In [2]:
datestamp = "20241024-100738"
sim_folder = pathlib.Path("/mnt/c/Users/nikko/OneDrive/Documents/Uni/magistrale/tesi/simulations")
datafolder = sim_folder / datestamp / "data"

In [3]:
path_list = []
for i,path in enumerate(datafolder.iterdir()):
    if path.is_dir():
        df_path = path / f"{datestamp}_run{i+1}.txt"
        path_list.append(df_path)

In [4]:
df_list = []
for path in path_list:
    df_list.append(pd.read_csv(path))

## Velocity calc.

I am dropping the first instant for velocity is obv. NaN. 

In [5]:
timestep = 1e-3

for df in df_list:
    df[["vx","vy"]] = df.groupby("N")[["xpos","ypos"]].diff()/timestep #velocity calculation
    df.dropna(inplace=True)
    df["Time"]=df["Time"]-1

In [6]:
df_list[0]

,N,Time,xpos,ypos,orientation,vx,vy
10,1,1,6.617691,4.903702,1.811750,27.331837,8.540876
11,2,1,3.886303,-7.938370,2.820682,-29.486347,40.782271
12,3,1,-1.993905,4.931552,-2.826403,12.468706,-6.830419
13,4,1,5.320242,9.639676,-2.816269,-6.220091,-39.012934
14,5,1,-8.879616,3.898408,-1.289558,-8.192026,-1.517515
...,...,...,...,...,...,...,...
10005,6,1000,5.573214,1.335729,-2.122243,-23.782125,-5.584749
10006,7,1000,-2.850001,-1.783798,-0.295584,16.782887,3.605311
10007,8,1000,-6.733105,4.961772,-2.559911,7.167954,2.648109
10008,9,1000,-11.908104,3.320195,-1.207984,-9.910789,-30.191820


In [7]:
num_exp = len(df_list)
Np = df_list[0].N.max()
num_steps = df_list[0].Time.max()

## Position reshaping

In [8]:
ppos = []
for df in df_list:
    Np = df.N.max()
    x = np.array(df.xpos).reshape(Np,-1)
    y = np.array(df.ypos).reshape(Np,-1)
    xy = np.stack([x,y],axis=2)
    ppos.append(xy)
# position = np.concatenate(ppos,axis=1)
position = np.stack(ppos,axis=3)
position = np.transpose(position,(3,1,0,2))


## Velocity reshaping

In [9]:
vv = []
for df in df_list:
    vx = np.array(df.vx).reshape(Np,-1)
    vy = np.array(df.vy).reshape(Np,-1)
    vxy = np.stack([vx,vy],axis=2)
    vv.append(vxy)
vel = np.stack(vv,axis=3)
vel = np.transpose(vel,(3,1,0,2))

## Drag Coefficient

In [10]:
R = 2.
gamma = 6*np.pi*R*1e-3
gamma = np.repeat(gamma, Np)
gamma = gamma[:,np.newaxis]
gamma = np.tile(gamma,(num_exp,num_steps,1,1))

## Edges

In [11]:
edges = list(itertools.combinations(range(Np), 2))
edges = np.array(edges)

In [12]:
data = {
    "position": position,
    "velocity": vel,
    "drag_coefficient": gamma,
    "edge_list": edges,
}

In [14]:
with open('data.pkl', '+rb') as f:
    pickle.dump(data, f)